In [1]:
import pandas as pd

def transformToNumber(currency_format):
    # transform $1.230.456,34 to 1230456.34
    currency_format = currency_format.replace(".", "").replace(",", ".").replace("$", "")
    return float(currency_format)

def load_dataframe(file_path):
    df = pd.read_csv(file_path, sep=";", encoding="latin1")
    # trim spaces on column names
    df.columns = df.columns.str.strip()
    df = df.dropna()
    df["Costo Contingente"] = df["Costo Contingente"].apply(transformToNumber)
    df["Month"] = df["Fecha Liquidación"].apply(lambda x: x.split("-")[1])
    df["Year"] = df["Fecha Liquidación"].apply(lambda x: x.split("-")[0])
    # Round to 2 decimal places
    df["Cost_per_beneficiary"] = (df["Costo Contingente"] / df["Beneficiarios"]).round(2)
    return df


data_2025_07 = load_dataframe("dataset/2025-07-pago-del-seguro-de-depositos.csv")



# Join two dataframes
data = data_2025_07.copy()

data.head()


,Ruc,Razón Social,Fecha Liquidación,Sector,Segmento,Provincia,Costo Contingente,Beneficiarios,Month,Year,Cost_per_beneficiary
3,9.906220e+11,"COOPERATIVA DE AHORRO Y CREDITO \ E.T.G.\"" LTD...",2019-01-24,COOP - SFPS,5,GUAYAS,51175.82,86.000,01,2019,595.07
4,4.915081e+11,COOPERATIVA DE AHORRO Y CREDITO 10 DE SEPTIEMB...,2019-12-11,COOP - SFPS,5,CARCHI,3042.82,21.000,12,2019,144.90
5,1.891708e+12,COOPERATIVA DE AHORRO Y CREDITO 15 DE DICIEMBR...,2016-05-16,COOP - SFPS,5,TUNGURAHUA,14458.24,461.000,05,2016,31.36
6,9.913137e+11,COOPERATIVA DE AHORRO Y CREDITO 19 DE SEPTIEMBRE,2018-06-21,COOP - SFPS,5,GUAYAS,72248.46,110.000,06,2018,656.80
7,1.891723e+12,COOPERATIVA DE AHORRO Y CREDITO 21 DE NOVIEMBR...,2020-07-01,COOP - SFPS,5,TUNGURAHUA,43302.50,1.132,07,2020,38253.09


In [2]:
# Remove atypical (outlier) values from data using outlier_mask and outlier_mask_costo
# Define outlier_mask and outlier_mask_costo using IQR method

def get_outlier_mask(series):
	Q1 = series.quantile(0.25)
	Q3 = series.quantile(0.75)
	IQR = Q3 - Q1
	return (series < (Q1 - 1.5 * IQR)) | (series > (Q3 + 1.5 * IQR))

outlier_mask = get_outlier_mask(data["Beneficiarios"])
outlier_mask_costo = get_outlier_mask(data["Costo Contingente"])

# Keep only rows that are not outliers in both columns
clean_data = data[~outlier_mask & ~outlier_mask_costo].copy()

data = clean_data.copy()

clean_data.head()

,Ruc,Razón Social,Fecha Liquidación,Sector,Segmento,Provincia,Costo Contingente,Beneficiarios,Month,Year,Cost_per_beneficiary
3,9.906220e+11,"COOPERATIVA DE AHORRO Y CREDITO \ E.T.G.\"" LTD...",2019-01-24,COOP - SFPS,5,GUAYAS,51175.82,86.000,01,2019,595.07
4,4.915081e+11,COOPERATIVA DE AHORRO Y CREDITO 10 DE SEPTIEMB...,2019-12-11,COOP - SFPS,5,CARCHI,3042.82,21.000,12,2019,144.90
5,1.891708e+12,COOPERATIVA DE AHORRO Y CREDITO 15 DE DICIEMBR...,2016-05-16,COOP - SFPS,5,TUNGURAHUA,14458.24,461.000,05,2016,31.36
6,9.913137e+11,COOPERATIVA DE AHORRO Y CREDITO 19 DE SEPTIEMBRE,2018-06-21,COOP - SFPS,5,GUAYAS,72248.46,110.000,06,2018,656.80
7,1.891723e+12,COOPERATIVA DE AHORRO Y CREDITO 21 DE NOVIEMBR...,2020-07-01,COOP - SFPS,5,TUNGURAHUA,43302.50,1.132,07,2020,38253.09


In [3]:
# Costo Beneficiario por Provincia

data_provincia = clean_data.groupby(["Provincia"])["Cost_per_beneficiary"].mean().round(2).reset_index()
data_provincia = data_provincia.sort_values("Cost_per_beneficiary", ascending=False)


In [4]:
# Media global
costo_promedio_global = data_provincia["Cost_per_beneficiary"].mean().round(2)
costo_promedio_global


np.float64(10271.52)

In [5]:

from ydata_profiling import ProfileReport

profile = ProfileReport(data, explorative=True)
profile.to_file("reporte.html")



Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [6]:
# create new dataframe where sum of Costo Contingente and beneficiarios by Year 
data_yearly = clean_data.groupby(["Year"])["Costo Contingente"].sum().reset_index()
data_yearly["Beneficiarios"] = clean_data.groupby(["Year"])["Beneficiarios"].sum().values
# Count the number of cooperativas by year
data_yearly["Total_Cooperativas"] = clean_data.groupby(["Year"]).size().values
data_yearly["costo promedio por beneficiario"] = (data_yearly["Costo Contingente"] / data_yearly["Beneficiarios"] )
data_yearly



,Year,Costo Contingente,Beneficiarios,Total_Cooperativas,costo promedio por beneficiario
0,2013,337016.33,1069.000,5,315.263171
1,2014,734924.29,1767.395,15,415.823452
2,2015,660262.35,2050.649,14,321.977262
3,2016,3205620.96,14620.402,75,219.256691
4,2017,1306677.22,2885.474,21,452.846645
5,2018,1340108.37,5447.482,17,246.005103
6,2019,479529.98,1023.235,9,468.641104
7,2020,685205.84,1001.381,11,684.260876
8,2021,405069.72,1803.779,11,224.567267
9,2022,682070.30,1878.064,8,363.177346


In [7]:
# Promedio global del costo por beneficiario
avg_global_costo_benef = data_yearly["costo promedio por beneficiario"].mean().round(2)
print("Promedio global del costo por beneficiario:", avg_global_costo_benef)

Promedio global del costo por beneficiario: 538.2


In [8]:
import matplotlib.pyplot as plt

# Provincia vs Costo Contingente
plt.figure(figsize=(12,6))
data.groupby('Provincia')['Costo Contingente'].sum().sort_values().plot(kind='bar')
plt.title('Costo Contingente por Provincia')
plt.ylabel('Costo Contingente')
plt.xlabel('Provincia')
plt.tight_layout()
plt.show()

# Provincia vs Beneficiarios
plt.figure(figsize=(12,6))
data.groupby('Provincia')['Beneficiarios'].sum().sort_values().plot(kind='bar')
plt.title('Beneficiarios por Provincia')
plt.ylabel('Beneficiarios')
plt.xlabel('Provincia')
plt.tight_layout()
plt.show()

# Provincia vs Segmento (conteo)
plt.figure(figsize=(12,6))
data.groupby(['Provincia', 'Segmento']).size().unstack().plot(kind='bar', stacked=True)
plt.title('Cantidad de Segmentos por Provincia')
plt.ylabel('Cantidad')
plt.xlabel('Provincia')
plt.tight_layout()
plt.show()

C:\Users\rober\AppData\Local\Temp\ipykernel_5904\2450961933.py:10: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()
C:\Users\rober\AppData\Local\Temp\ipykernel_5904\2450961933.py:19: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()
C:\Users\rober\AppData\Local\Temp\ipykernel_5904\2450961933.py:28: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()
C:\Users\rober\AppData\Local\Temp\ipykernel_5904\2450961933.py:28: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [9]:
import seaborn as sns
import matplotlib.pyplot as plt


# detección de valores atípicos
numerical_col = data.select_dtypes(include=['int', 'float']).columns
plt.figure
fig, ax = plt.subplots(1,2, figsize=(20,15))

# tenemos 4 valores numéricos.
row, col = 1,2
col_count = 0
for c in range(col):
    if col_count >= len(numerical_col):
        ax[c].text(0.5, 0.5, "no data")
    else:
        sns.histplot(data=data, x=numerical_col[col_count], ax=ax[c])
    col_count += 1

# sns and plt display on notebook


In [10]:
from sklearn.preprocessing import LabelEncoder

data_kmeans = data.copy()

# Etiquetado de variables categoricas como Provincia use Label Encoder

le = LabelEncoder()
data_kmeans['Provincia'] = le.fit_transform(data_kmeans['Provincia'])
data_kmeans["Cooperativa"] = le.fit_transform(data_kmeans["Razón Social"])

data_kmeans.head()


,Ruc,Razón Social,Fecha Liquidación,Sector,Segmento,Provincia,Costo Contingente,Beneficiarios,Month,Year,Cost_per_beneficiary,Cooperativa
3,9.906220e+11,"COOPERATIVA DE AHORRO Y CREDITO \ E.T.G.\"" LTD...",2019-01-24,COOP - SFPS,5,10,51175.82,86.000,01,2019,595.07,188
4,4.915081e+11,COOPERATIVA DE AHORRO Y CREDITO 10 DE SEPTIEMB...,2019-12-11,COOP - SFPS,5,4,3042.82,21.000,12,2019,144.90,0
5,1.891708e+12,COOPERATIVA DE AHORRO Y CREDITO 15 DE DICIEMBR...,2016-05-16,COOP - SFPS,5,22,14458.24,461.000,05,2016,31.36,1
6,9.913137e+11,COOPERATIVA DE AHORRO Y CREDITO 19 DE SEPTIEMBRE,2018-06-21,COOP - SFPS,5,10,72248.46,110.000,06,2018,656.80,2
7,1.891723e+12,COOPERATIVA DE AHORRO Y CREDITO 21 DE NOVIEMBR...,2020-07-01,COOP - SFPS,5,22,43302.50,1.132,07,2020,38253.09,3


In [11]:
col = ["Costo Contingente", "Beneficiarios", "Provincia","Segmento"]
sns.pairplot(data=data_kmeans[col], hue="Segmento")

# Análisis DEA (CCR-output) para eficiencia operativa
El Análisis Envolvente de Datos (DEA) permite evaluar la eficiencia relativa de unidades de decisión (DMUs), como provincias u oficinas, considerando múltiples insumos y productos. El modelo CCR orientado a output identifica las unidades más eficientes y sirve como referencia para mejorar la gestión y asignación de recursos.

In [ ]:
# DEA CCR-output: Evaluación de eficiencia por provincia
import pandas as pd
from pyDEA.core.utils.model_builder import build_models
from pyDEA.core.data_processing.parameters import Parameters
from pyDEA.core.data_processing.input_data import InputData
import matplotlib.pyplot as plt
# Preparar datos para DEA: DMU = Provincia, Input = Beneficiarios, Output = Costo Contingente
dea_data = data.groupby('Provincia').agg({'Beneficiarios':'sum','Costo Contingente':'sum'}).reset_index()
dea_data = dea_data.rename(columns={'Provincia':'DMU'})
# Crear InputData correctamente desde DataFrame
input_categories = ['Beneficiarios']
output_categories = ['Costo Contingente']
input_data = InputData()
input_data.read_data_from_data_frame(dea_data, dmu_column='DMU', input_categories=input_categories, output_categories=output_categories)
# Definir parámetros del modelo DEA CCR-output
params = Parameters()
params.params = {
    'orientation': 'output',
    'returns': 'CRS',
    'input_vars': input_categories,
    'output_vars': output_categories,
    'INPUT_CATEGORIES': input_categories,
    'OUTPUT_CATEGORIES': output_categories
}
# Construir y ejecutar el modelo DEA
models = build_models(params=params, model_input=input_data)
for m in models:
    result = m.run()
    efficiency = result.efficiency_scores
    eff_df = pd.DataFrame({'Provincia': dea_data['DMU'], 'Eficiencia CCR-output': efficiency})
    print(eff_df)
    plt.figure(figsize=(12,6))
    plt.bar(eff_df['Provincia'], eff_df['Eficiencia CCR-output'])
    plt.xticks(rotation=90)
    plt.title('Eficiencia CCR-output por Provincia')
    plt.ylabel('Eficiencia')
    plt.xlabel('Provincia')
    plt.tight_layout()
    plt.show()

KeyError: 'Beneficiarios is not present in categories list'